In [9]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
from sklearn import model_selection
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [10]:
path = "C:\\Users\\Ivand\\Repos\\banks\\data"

In [11]:
df = pd.read_csv(path + "\\BanksDFUsable.csv")
df = df.drop(df.columns[0], axis = 1)
df_ratings = pd.read_csv(path + "\\RatingsClean.csv")
df_normatives = pd.read_csv(path + "\\NormativesUsable.csv")
df_normatives.drop(df_normatives.columns[0], axis = 1, inplace = True)

In [12]:
df = df[(df.Year >= 2015)&(df.Year <= 2020)]

In [13]:
#count non-zeros in each column
df.astype(bool).sum(axis = 0).sort_values()

BadBorrowing_Me               1280
Revaluation_CBR               2796
DefaultIn365Days              3637
CBRCredits_CBR                6090
DerivativeLiabilities_CBR     6105
                             ...  
Equity                       40700
REGN                         40702
Year                         40702
Month                        40702
Date                         40702
Length: 62, dtype: int64

In [14]:
df_ratings = df_ratings.rename(columns = {"regn" : "REGN"})

In [15]:
df.REGN = df.REGN.apply(str)
df_ratings.REGN = df_ratings.REGN.apply(str)

In [16]:
df_ratings[df_ratings.REGN == "1"]

,Moodys,name,ExpertRA,short_name,form,REGN,region,ownership,personal_page,app,Year,Month,foreign
31345,no,ЮниКредит Банк,no,ЮниКредит Банк,private,1,moscow,private,1,1,2015,1,1.0
31346,no,ЮниКредит Банк,no,ЮниКредит Банк,private,1,moscow,private,1,1,2015,2,1.0
31347,no,ЮниКредит Банк,no,ЮниКредит Банк,private,1,moscow,private,1,1,2015,3,1.0
31348,no,ЮниКредит Банк,no,ЮниКредит Банк,private,1,moscow,private,1,1,2015,4,1.0
31349,no,ЮниКредит Банк,no,ЮниКредит Банк,private,1,moscow,private,1,1,2015,5,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
31412,no,ЮниКредит Банк,ruAAA,ЮниКредит Банк,private,1,moscow,private,1,1,2020,8,1.0
31413,no,ЮниКредит Банк,ruAAA,ЮниКредит Банк,private,1,moscow,private,1,1,2020,9,1.0
31414,no,ЮниКредит Банк,ruAAA,ЮниКредит Банк,private,1,moscow,private,1,1,2020,10,1.0
31415,no,ЮниКредит Банк,ruAAA,ЮниКредит Банк,private,1,moscow,private,1,1,2020,11,1.0


In [17]:
df_ratings.REGN = df_ratings.REGN.apply(lambda x: re.sub("-К", "", x))
df_ratings.REGN = df_ratings.REGN.apply(lambda x: re.sub("-Д", "", x))
df_ratings.REGN = df_ratings.REGN.apply(lambda x: re.sub("-С", "", x))
df_ratings.REGN = df_ratings.REGN.apply(lambda x: re.sub("-Г", "", x))
df_ratings.REGN = df_ratings.REGN.apply(lambda x: re.sub("-Р", "", x))
df_normatives.REGN = df_normatives.REGN.apply(str)

df_full = df.merge(df_ratings, on = ["Year", "Month", "REGN"], how = "left")
#этот фильтр удаляет только национальный клиринговый центр
df_full = df_full[df_full.short_name == df_full.short_name]
df_full = df_full.merge(df_normatives, on = ["REGN", "Year", "Month"], how = "left")
df_full = df_full.fillna(0)

In [18]:
#df_full.to_csv("C:\\Users\\Ivand\\Repos\\banks\\data\\BanksDFFull.csv")

In [19]:
df_full.columns

Index(['REGN', 'Year', 'Month', 'NetIncome', 'NetSecurityIncome',
       'NetDerivativeIncome', 'NetFeeIncome', 'NetInvestmentIncome',
       'NetInterestIncome', 'Assets', 'Liabilities', 'Equity', 'Securities',
       'Investment', 'RetainedEarning', 'DefaultIn365Days', 'DefaultIn730Days',
       'DefaultIn10000Days', 'Reserves_Me', 'LendCorporate_Me', 'LendRetail',
       'BorrowCorp_Me', 'BorrowRetail_Me', 'LendInterbank_Me',
       'BorrowInterbank_Me', 'LendShort_Me', 'BorrowShort_Me', 'LendLong_Me',
       'BorrowLong_Me', 'Cash_Me', 'BondsIssued_Me', 'BadCredits_Me',
       'LendState_Me', 'BorrowState_Me', 'CurrentAccounts_Me',
       'TradingAccounts_Me', 'BadBorrowing_Me', 'Unclassified',
       'CBRDeposits_CBR', 'ObligatoryReserves_CBR', 'CreditsToBanks_CBR',
       'Securities_CBR', 'ShareCapitalParticipation_CBR',
       'CreditPortfolio_CBR', 'ProfitableDerivatives_CBR', 'CoreFunds_CBR',
       'Intangibles_CBR', 'DeferredTaxAsset_CBR', 'OtherAssets_CBR',
       'CBRCred

In [20]:
df_full.isna().any()

REGN                                 False
Year                                 False
Month                                False
NetIncome                            False
NetSecurityIncome                    False
                                     ...  
CreditsToInsidersCoefficient_norm    False
MarketRisk0_norm                     False
InterestRisk0_norm                   False
MarketRiskII_norm                    False
InterestRiskII_norm                  False
Length: 98, dtype: bool

In [21]:
df_full = df_full[df_full.Assets != 0]
df_full = df_full[df_full.NetIncome != 0]


In [22]:
divide_by_net_income = ["NetSecurityIncome", 'NetDerivativeIncome',
         'NetFeeIncome', 'NetInvestmentIncome',
         'NetInterestIncome']
df_full.loc[:, divide_by_net_income] = df_full.loc[:, divide_by_net_income].div(df_full["NetIncome"], axis=0)

divide_by_assets = ["NetIncome", 'Liabilities', 'Equity', 'Securities',
       'Investment', 'RetainedEarning','Reserves_Me', 'LendCorporate_Me', 'LendRetail',
       'BorrowCorp_Me', 'BorrowRetail_Me', 'LendInterbank_Me',
       'BorrowInterbank_Me', 'LendShort_Me', 'BorrowShort_Me', 'LendLong_Me',
       'BorrowLong_Me', 'Cash_Me', 'BondsIssued_Me', 'BadCredits_Me',
       'LendState_Me', 'BorrowState_Me', 'CurrentAccounts_Me',
       'TradingAccounts_Me', 'BadBorrowing_Me', 'Unclassified',
       'CBRDeposits_CBR', 'ObligatoryReserves_CBR', 'CreditsToBanks_CBR',
       'Securities_CBR', 'ShareCapitalParticipation_CBR',
       'CreditPortfolio_CBR', 'ProfitableDerivatives_CBR', 'CoreFunds_CBR',
       'Intangibles_CBR', 'DeferredTaxAsset_CBR', 'OtherAssets_CBR',
       'CBRCredits_CBR', 'BanksFunds_CBR', 'ClientFunds_CBR',
       'SecuritiesIssued_CBR', 'DerivativeLiabilities_CBR',
       'OtherLiabilities_CBR', 'Revaluation_CBR', 'MainEquity_CBR',
       'ExtraEquity_CBR', 'ReserveFund_CBR', 'SecuritiesRevaluation_CBR',
       'RetainedEarnings_CBR', 'LiquidAssets_norm',
       'RiskFree_norm', 'RiskWeightedI0_norm', 'RiskWeightedIV0_norm','RiskWeightedII0_norm', 'LiquidityMonth_norm',
       'LiabilitiesOnDemandT_norm', 'HighlyLiquidAssets_norm','RiskyOperations0_norm',
       'RiskyOperationsII_norm', 'LiquidityDay_norm',
       'LiabilitiesOnDemandM_norm',"ShortCreditAssets_norm", 'RiskyOperationsI_norm', 'CreditLiabilities_norm','MarketRisk0_norm',
       'InterestRisk0_norm', 'MarketRiskII_norm', 'InterestRiskII_norm']
df_full.loc[:, divide_by_assets] = df_full.loc[:, divide_by_assets].div(df_full["Assets"], axis=0)

In [23]:
df_full.columns[df_full.isna().any()]

Index([], dtype='object')

In [24]:
factors_list = ['REGN', 'Year', 'Month', 'ownership', 'personal_page', 'app', 'foreign']

In [25]:
df_full.drop(['DefaultIn365Days', 'DefaultIn730Days',
       'DefaultIn10000Days', "Date", "short_name", "name"], axis = 1, inplace = True)

In [26]:
#we can see that the least populated class in Moodys (Ca) has only three features. Lets group all the Ca*
#classes to one class "C"
df_full.groupby(df_full.Moodys).count()


,REGN,Year,Month,NetIncome,NetSecurityIncome,NetDerivativeIncome,NetFeeIncome,NetInvestmentIncome,NetInterestIncome,Assets,...,ShortCreditAssets_norm,LargeRiskCoefficient_norm,LiquidityLong_norm,RiskyOperationsI_norm,CreditLiabilities_norm,CreditsToInsidersCoefficient_norm,MarketRisk0_norm,InterestRisk0_norm,MarketRiskII_norm,InterestRiskII_norm
Moodys,,,,,,,,,,,,,,,,,,,,,
B1,430,430,430,430,430,430,430,430,430,430,...,430,430,430,430,430,430,430,430,430,430
B2,665,665,665,665,665,665,665,665,665,665,...,665,665,665,665,665,665,665,665,665,665
B3,700,700,700,700,700,700,700,700,700,700,...,700,700,700,700,700,700,700,700,700,700
Ba1,452,452,452,452,452,452,452,452,452,452,...,452,452,452,452,452,452,452,452,452,452
Ba2,392,392,392,392,392,392,392,392,392,392,...,392,392,392,392,392,392,392,392,392,392
Ba3,421,421,421,421,421,421,421,421,421,421,...,421,421,421,421,421,421,421,421,421,421
Baa3,130,130,130,130,130,130,130,130,130,130,...,130,130,130,130,130,130,130,130,130,130
Ca,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
Caa1,99,99,99,99,99,99,99,99,99,99,...,99,99,99,99,99,99,99,99,99,99


In [27]:
df_full.loc[(df_full.Moodys == "Ca") | \
             (df_full.Moodys == "Caa1") | \
             (df_full.Moodys == "Caa2") | \
             (df_full.Moodys == "Caa3"), "Moodys"] = "C"

In [28]:
#with ExpertRA the situation is even more critical; the number of discrete classes is extremely large. Lets 
#group them in bigger classes
df_full.groupby(df_full.ExpertRA).count()

,REGN,Year,Month,NetIncome,NetSecurityIncome,NetDerivativeIncome,NetFeeIncome,NetInvestmentIncome,NetInterestIncome,Assets,...,ShortCreditAssets_norm,LargeRiskCoefficient_norm,LiquidityLong_norm,RiskyOperationsI_norm,CreditLiabilities_norm,CreditsToInsidersCoefficient_norm,MarketRisk0_norm,InterestRisk0_norm,MarketRiskII_norm,InterestRiskII_norm
ExpertRA,,,,,,,,,,,,,,,,,,,,,
A (I),511,511,511,511,511,511,511,511,511,511,...,511,511,511,511,511,511,511,511,511,511
A (II),469,469,469,469,469,469,469,469,469,469,...,469,469,469,469,469,469,469,469,469,469
A (III),662,662,662,662,662,662,662,662,662,662,...,662,662,662,662,662,662,662,662,662,662
A+,6,6,6,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6
A+ (I),175,175,175,175,175,175,175,175,175,175,...,175,175,175,175,175,175,175,175,175,175
A+ (II),117,117,117,117,117,117,117,117,117,117,...,117,117,117,117,117,117,117,117,117,117
A+ (III),291,291,291,291,291,291,291,291,291,291,...,291,291,291,291,291,291,291,291,291,291
A++,159,159,159,159,159,159,159,159,159,159,...,159,159,159,159,159,159,159,159,159,159
B,49,49,49,49,49,49,49,49,49,49,...,49,49,49,49,49,49,49,49,49,49


In [29]:
df_full.loc[(df_full.ExpertRA == "ruC") | \
             (df_full.ExpertRA == "ruCC") | \
             (df_full.ExpertRA == "ruCCC"), "ExpertRA"] = "ruC"
df_full.loc[(df_full.ExpertRA == "A+"), "ExpertRA"] = "A+ (I)"
df_full.loc[(df_full.ExpertRA == "A (I)") | \
             (df_full.ExpertRA == "A (II)") | \
             (df_full.ExpertRA == "A (III)"), "ExpertRA"] = "A"
df_full.loc[(df_full.ExpertRA == "A+ (I)") | \
             (df_full.ExpertRA == "A+ (II)") | \
             (df_full.ExpertRA == "A+ (III)"), "ExpertRA"] = "A+"
df_full.loc[(df_full.ExpertRA == "A++ (I)") | \
             (df_full.ExpertRA == "A++ (II)") | \
             (df_full.ExpertRA == "A++ (III)"), "ExpertRA"] = "A++"
df_full.loc[(df_full.ExpertRA == "С") | \
             (df_full.ExpertRA == "C+") | \
             (df_full.ExpertRA == "C++") | \
             (df_full.ExpertRA == "E"), "ExpertRA"] = "C"
df_full.loc[(df_full.ExpertRA == "ruAA") | \
             (df_full.ExpertRA == "ruAA+") | \
             (df_full.ExpertRA == "ruAA-"), "ExpertRA"] = "ruAA"
df_full.loc[(df_full.ExpertRA == "B") | \
             (df_full.ExpertRA == "B+") | \
             (df_full.ExpertRA == "B++"), "ExpertRA"] = "B"
df_full.loc[(df_full.ExpertRA == "A") | \
             (df_full.ExpertRA == "A+") | \
             (df_full.ExpertRA == "A++"), "ExpertRA"] = "A"
df_full.loc[(df_full.ExpertRA == "ruA") | \
             (df_full.ExpertRA == "ruA+") | \
             (df_full.ExpertRA == "ruA-"), "ExpertRA"] = "ruA"
df_full.loc[(df_full.ExpertRA == "ruB") | \
             (df_full.ExpertRA == "ruB+") | \
             (df_full.ExpertRA == "ruB-"), "ExpertRA"] = "ruB"
df_full.loc[(df_full.ExpertRA == "ruBB") | \
             (df_full.ExpertRA == "ruBB+") | \
             (df_full.ExpertRA == "ruBB-"), "ExpertRA"] = "ruBB"
df_full.loc[(df_full.ExpertRA == "ruBBB") | \
             (df_full.ExpertRA == "ruBBB+") | \
             (df_full.ExpertRA == "ruBBB-"), "ExpertRA"] = "ruBBB"

In [30]:
df_full = pd.get_dummies(df_full, columns = ["ownership", "form", "region"], drop_first = True)

In [31]:
df_with_Moodys = df_full[df_full.Moodys != "no"]
df_with_Expertra = df_full[df_full.ExpertRA != "no"]

In [79]:
X_Moodys = df_with_Moodys.drop(["Moodys", "ExpertRA"], axis = 1)
X_Expertra = df_with_Expertra.drop(["Moodys", "ExpertRA"], axis = 1)

Moodys = df_with_Moodys["Moodys"]
Expertra = df_with_Expertra["ExpertRA"]

X_Moodys_train, X_Moodys_test, Moodys_train, Moodys_test =\
                    model_selection.train_test_split(X_Moodys, Moodys, test_size=0.2)
X_Expertra_train, X_Expertra_test, Expertra_train, Expertra_test =\
                    model_selection.train_test_split(X_Expertra, Expertra, test_size=0.2)

X_Moodys_train.reset_index(inplace=True, drop=True)
X_Moodys_test.reset_index(inplace=True, drop=True)
X_Expertra_train.reset_index(inplace=True, drop=True)
X_Expertra_test.reset_index(inplace=True, drop=True)
Moodys_train.reset_index(inplace=True, drop=True)
Moodys_test.reset_index(inplace=True, drop=True)
Expertra_train.reset_index(inplace=True, drop=True)
Expertra_test.reset_index(inplace=True, drop=True)

Train-test split with non-overlapping banks for KNN and similar algorithms

In [66]:
df_with_Moodys_banks = df_with_Moodys.REGN
df_with_Moodys_banks = df_with_Moodys_banks.unique()
df_with_Moodys_banks_test = np.random.choice(df_with_Moodys_banks, round(len(df_with_Moodys_banks)/5))

Full_Moodys_test_KNN = df_with_Moodys[df_with_Moodys.REGN.isin(df_with_Moodys_banks_test)]
Full_Moodys_train_KNN = df_with_Moodys[~df_with_Moodys.REGN.isin(df_with_Moodys_banks_test)]

X_Moodys_test_KNN = Full_Moodys_test_KNN.drop(["Moodys", "ExpertRA"], axis = 1)
X_Moodys_train_KNN = Full_Moodys_train_KNN.drop(["Moodys", "ExpertRA"], axis = 1)
Moodys_test_KNN = Full_Moodys_test_KNN["Moodys"]
Moodys_train_KNN = Full_Moodys_train_KNN["Moodys"]

X_Moodys_train_KNN.reset_index(inplace=True, drop=True)
X_Moodys_test_KNN.reset_index(inplace=True, drop=True)
Moodys_train_KNN.reset_index(inplace=True, drop=True)
Moodys_test_KNN.reset_index(inplace=True, drop=True)

In [70]:
Moodys_train_KNN

0       Ba1
1       Ba1
2       Ba1
3       Ba1
4       Ba1
       ... 
3541    Ba2
3542    Ba2
3543    Ba2
3544    Ba2
3545    Ba2
Name: Moodys, Length: 3546, dtype: object

# Moodys 

The code below performs feature selection. I do not recommend to reproduce it, since it takes long (about 2 hours). Hard-coded list of features is copypasted from output.

In [37]:
short_feature_list = ['RetainedEarning', 'Reserves_Me', 'LendCorporate_Me', 'BorrowShort_Me',
                      'LendLong_Me', 'BorrowLong_Me', 'Cash_Me', 'BadBorrowing_Me', 
                      'ObligatoryReserves_CBR', 'CoreFunds_CBR', 'Intangibles_CBR',
                      'ClientFunds_CBR', 'ExtraEquity_CBR', 'SecuritiesRevaluation_CBR',
                      'RiskWeightedIV0_norm', 'LiquidityMonth_norm', 'LiquidityDay_norm',
                      'InterestRisk0_norm', 'InterestRiskII_norm', 'form_public']

### Moodys Logistic Regression

In [38]:
estimator=LogisticRegression(penalty="none", max_iter=1000, n_jobs = 4)
estimator.fit(X_Moodys_train, Moodys_train)
estimator.score(X_Moodys_test, Moodys_test)

0.09582863585118377

In [84]:
estimator=LogisticRegression(penalty="none", max_iter=1000, n_jobs = 4)
estimator.fit(X_Moodys_train[short_feature_list], Moodys_train)
estimator.score(X_Moodys_test[short_feature_list], Moodys_test)

0.5636978579481398

In [89]:
comparizon_approximate(Moodys_test, X_Moodys_test[short_feature_list], estimator, short_feature_list)

0.6820744081172492

In [99]:
def comparizon_approximate(Moodys_test, X_Moodys_test, estimator, feature_list = None):
    if feature_list == None:
        compare = pd.DataFrame({"Real" : Moodys_test,
                  "Predicted" : estimator.predict(X_Moodys_test)})
    else:
        compare = pd.DataFrame({"Real" : Moodys_test,
                  "Predicted" : estimator.predict(X_Moodys_test[feature_list])})

    compare.Real = compare.Real.map({"B1" : 0,
    "B2" : 1,
    "B3" : 2,
    "Ba1" : 3, 
    "Ba2" : 4, 
    "Ba3" : 5, 
    "Baa3" : 6, 
    "C" : 7,
    "withdrawn" : 8})
    compare.Predicted = compare.Predicted.map({"B1" : 0,
    "B2" : 1,
    "B3" : 2,
    "Ba1" : 3, 
    "Ba2" : 4, 
    "Ba3" : 5, 
    "Baa3" : 6, 
    "C" : 7,
    "withdrawn" : 8})



    compare["Error"] = compare.Real-compare.Predicted

    compare.groupby("Error").count()

    return compare[(compare.Error == 0)|\
           (compare.Error == 1)|\
            (compare.Error == -1)].count()[0]/compare.count()[0]
    

In [42]:
from sklearn.neighbors import KNeighborsClassifier


In [122]:
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
# classifier.fit(X_Moodys_train_KNN, Moodys_train_KNN)
# comparizon_approximate(Moodys_test_KNN, X_Moodys_test_KNN, classifier, X_Moodys_test_KNN.columns)
classifier.fit(X_Moodys_train_KNN[short_feature_list], Moodys_train_KNN)
comparizon_approximate(Moodys_test_KNN, X_Moodys_test_KNN[short_feature_list], classifier, short_feature_list)

0.48645598194130923

0.48645598194130923

In [108]:
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing

scaler = preprocessing.StandardScaler().fit(X_Moodys_train_KNN[short_feature_list])
X_Moodys_train_NN = scaler.transform(X_Moodys_train_KNN[short_feature_list])
scaler_test = preprocessing.StandardScaler().fit(X_Moodys_test_KNN[short_feature_list])
X_Moodys_test_NN = scaler.transform(X_Moodys_test_KNN[short_feature_list])

clf = MLPClassifier(solver='lbfgs')
clf.fit(X_Moodys_train_NN, Moodys_train_KNN)
comparizon_approximate(Moodys_test_KNN, X_Moodys_test_NN, clf)

0.4898419864559819

In [112]:
parameter_space = {
    'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam', 'lbfgs'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}

In [113]:
from sklearn.model_selection import GridSearchCV

clf = GridSearchCV(MLPClassifier(), parameter_space, n_jobs=-1, cv=3)
clf.fit(X_Moodys_train_NN, Moodys_train_KNN)
comparizon_approximate(Moodys_test_KNN, X_Moodys_test_NN, clf)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


0.5338600451467269

In [123]:
df_full["MoodysPredicted"] = estimator.predict(df_full[short_feature_list])

In [124]:
df_full["NoMoodys"] = 0
df_full.loc[df_full.Moodys == "no", "NoMoodys"] = 1

In [125]:
df_full.loc[df_full.Moodys == "no", "Moodys"] = df_full.loc[df_full.Moodys == "no", "MoodysPredicted"]

In [126]:
df_full.drop(["MoodysPredicted"], axis = 1).to_csv("C:\\Users\\Ivand\\Repos\\banks\\data\\BanksDFPrepared.csv")

# ExpertRA

In [123]:
X_Expertra.Assets = (X_Expertra.Assets - X_Expertra.Assets.mean())/(X_Expertra.Assets.std(ddof = 0))

Even having been normalized, the selection for expertra rating converges only at very lots of iterations, which takes a lot of time. Further I will use the same short list of most useful variables as for Moodys.

### Expertra Logistic Regression

In [145]:
estimator=LogisticRegression(penalty="none", max_iter=1000, n_jobs = 4)
estimator.fit(X_Expertra_train, Expertra_train)
estimator.score(X_Expertra_test, Expertra_test)

0.01057340382269215

In [147]:
estimator=LogisticRegression(penalty="none", max_iter=1000, n_jobs = 4)
estimator.fit(X_Expertra_train[short_feature_list], Expertra_train)
estimator.score(X_Expertra_test[short_feature_list], Expertra_test)

0.3607157381049207

# Defaults

In [164]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
from sklearn import model_selection
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LogisticRegression

In [165]:
path = "C:\\Users\\Ivand\\Repos\\banks\\data"

In [166]:
df = pd.read_csv(path + "\\BanksDFUsable.csv")
df = df.drop(df.columns[0], axis = 1)
df_ratings = pd.read_csv(path + "\\RatingsClean.csv")

In [167]:
df = df[(df.Year >= 2015)&(df.Year <= 2019)]

In [168]:
#count non-zeros in each column
df.astype(bool).sum(axis = 0).sort_values()

BadBorrowing_Me               1050
Revaluation_CBR               1405
DefaultIn365Days              3397
DerivativeLiabilities_CBR     5085
CBRCredits_CBR                5379
                             ...  
Equity                       35724
REGN                         35726
Year                         35726
Month                        35726
Date                         35726
Length: 62, dtype: int64

In [169]:
df_ratings = df_ratings.rename(columns = {"regn" : "REGN"})

In [170]:
df.REGN = df.REGN.apply(str)
df_ratings.REGN = df_ratings.REGN.apply(str)

In [171]:
df_ratings[df_ratings.REGN == "1"]

,Moodys,name,ExpertRA,short_name,form,REGN,region,ownership,personal_page,app,Year,Month,foreign
31345,no,ЮниКредит Банк,no,ЮниКредит Банк,private,1,moscow,private,1,1,2015,1,1.0
31346,no,ЮниКредит Банк,no,ЮниКредит Банк,private,1,moscow,private,1,1,2015,2,1.0
31347,no,ЮниКредит Банк,no,ЮниКредит Банк,private,1,moscow,private,1,1,2015,3,1.0
31348,no,ЮниКредит Банк,no,ЮниКредит Банк,private,1,moscow,private,1,1,2015,4,1.0
31349,no,ЮниКредит Банк,no,ЮниКредит Банк,private,1,moscow,private,1,1,2015,5,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
31412,no,ЮниКредит Банк,ruAAA,ЮниКредит Банк,private,1,moscow,private,1,1,2020,8,1.0
31413,no,ЮниКредит Банк,ruAAA,ЮниКредит Банк,private,1,moscow,private,1,1,2020,9,1.0
31414,no,ЮниКредит Банк,ruAAA,ЮниКредит Банк,private,1,moscow,private,1,1,2020,10,1.0
31415,no,ЮниКредит Банк,ruAAA,ЮниКредит Банк,private,1,moscow,private,1,1,2020,11,1.0


In [172]:
df_ratings.REGN = df_ratings.REGN.apply(lambda x: re.sub("-К", "", x))
df_ratings.REGN = df_ratings.REGN.apply(lambda x: re.sub("-Д", "", x))
df_ratings.REGN = df_ratings.REGN.apply(lambda x: re.sub("-С", "", x))
df_ratings.REGN = df_ratings.REGN.apply(lambda x: re.sub("-Г", "", x))
df_ratings.REGN = df_ratings.REGN.apply(lambda x: re.sub("-Р", "", x))

df_full = df.merge(df_ratings, on = ["Year", "Month", "REGN"], how = "left")
#этот фильтр удаляет только национальный клиринговый центр
df_full = df_full[df_full.short_name == df_full.short_name]
df_full

,REGN,Year,Month,NetIncome,NetSecurityIncome,NetDerivativeIncome,NetFeeIncome,NetInvestmentIncome,NetInterestIncome,Assets,...,Moodys,name,ExpertRA,short_name,form,region,ownership,personal_page,app,foreign
0,1,2015,1,8872670.0,-711540.0,16159570.0,8903905.0,1.0,28990932.0,1.209364e+09,...,no,ЮниКредит Банк,no,ЮниКредит Банк,private,moscow,private,1.0,1.0,1.0
1,1,2015,2,8872670.0,-711540.0,16159570.0,8903905.0,1.0,28990932.0,1.328361e+09,...,no,ЮниКредит Банк,no,ЮниКредит Банк,private,moscow,private,1.0,1.0,1.0
2,1,2015,3,8872670.0,-711540.0,16159570.0,8903905.0,1.0,28990932.0,1.203223e+09,...,no,ЮниКредит Банк,no,ЮниКредит Банк,private,moscow,private,1.0,1.0,1.0
3,1,2015,4,2586956.0,-194556.0,3375908.0,1570702.0,-0.0,4807043.0,1.167734e+09,...,no,ЮниКредит Банк,no,ЮниКредит Банк,private,moscow,private,1.0,1.0,1.0
4,1,2015,5,2586956.0,-194556.0,3375908.0,1570702.0,-0.0,4807043.0,1.072174e+09,...,no,ЮниКредит Банк,no,ЮниКредит Банк,private,moscow,private,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35803,992,2016,8,51261.0,85340.0,66247.0,8122.0,-0.0,17486.0,2.201881e+06,...,no,Вологдабанк,no,Вологдабанк,private,notmoscow,private,0.0,0.0,0.0
35804,992,2016,9,51261.0,85340.0,66247.0,8122.0,-0.0,17486.0,2.233772e+06,...,no,Вологдабанк,no,Вологдабанк,private,notmoscow,private,0.0,0.0,0.0
35805,992,2016,10,37506.0,156479.0,84845.0,13325.0,-14.0,-47470.0,2.240207e+06,...,no,Вологдабанк,no,Вологдабанк,private,notmoscow,private,0.0,0.0,0.0
35806,992,2016,11,37506.0,156479.0,84845.0,13325.0,-14.0,-47470.0,2.064643e+06,...,no,Вологдабанк,no,Вологдабанк,private,notmoscow,private,0.0,0.0,0.0


In [173]:
df_full = df_full[df_full.Assets != 0]
df_full = df_full[df_full.NetIncome != 0]


In [174]:
divide_by_net_income = ["NetSecurityIncome", 'NetDerivativeIncome',
         'NetFeeIncome', 'NetInvestmentIncome',
         'NetInterestIncome']
df_full.loc[:, divide_by_net_income] = df_full.loc[:, divide_by_net_income].div(df_full["NetIncome"], axis=0)

divide_by_assets = ["NetIncome", 'Liabilities', 'Equity', 'Securities',
       'Investment', 'RetainedEarning','Reserves_Me', 'LendCorporate_Me', 'LendRetail',
       'BorrowCorp_Me', 'BorrowRetail_Me', 'LendInterbank_Me',
       'BorrowInterbank_Me', 'LendShort_Me', 'BorrowShort_Me', 'LendLong_Me',
       'BorrowLong_Me', 'Cash_Me', 'BondsIssued_Me', 'BadCredits_Me',
       'LendState_Me', 'BorrowState_Me', 'CurrentAccounts_Me',
       'TradingAccounts_Me', 'BadBorrowing_Me', 'Unclassified',
       'CBRDeposits_CBR', 'ObligatoryReserves_CBR', 'CreditsToBanks_CBR',
       'Securities_CBR', 'ShareCapitalParticipation_CBR',
       'CreditPortfolio_CBR', 'ProfitableDerivatives_CBR', 'CoreFunds_CBR',
       'Intangibles_CBR', 'DeferredTaxAsset_CBR', 'OtherAssets_CBR',
       'CBRCredits_CBR', 'BanksFunds_CBR', 'ClientFunds_CBR',
       'SecuritiesIssued_CBR', 'DerivativeLiabilities_CBR',
       'OtherLiabilities_CBR', 'Revaluation_CBR', 'MainEquity_CBR',
       'ExtraEquity_CBR', 'ReserveFund_CBR', 'SecuritiesRevaluation_CBR',
       'RetainedEarnings_CBR']
df_full.loc[:, divide_by_assets] = df_full.loc[:, divide_by_assets].div(df_full["Assets"], axis=0)

In [175]:
df_full.drop(["Date", "short_name", "name"], axis = 1, inplace = True)

In [178]:
df_full = pd.get_dummies(df_full, columns = ["ownership", "form", "region"], drop_first = True)

In [183]:
seed = 42
X = df_full.drop(['DefaultIn365Days', 'DefaultIn730Days',
       'DefaultIn10000Days'], axis = 1)
X.drop(["Moodys", "ExpertRA"], axis = 1, inplace = True)
OneYear = df_full['DefaultIn365Days']
TwoYears = df_full["DefaultIn730Days"]
AllYears = df_full["DefaultIn10000Days"]

X_train, X_test, OneYear_train, OneYear_test, TwoYears_train, TwoYears_test, AllYears_train, AllYears_test =\
                    model_selection.train_test_split(X, OneYear, TwoYears, AllYears, test_size=0.2, random_state=seed)


X_train.reset_index(inplace=True, drop=True)
X_test.reset_index(inplace=True, drop=True)
OneYear_train.reset_index(inplace=True, drop=True)
OneYear_test.reset_index(inplace=True, drop=True)
TwoYears_train.reset_index(inplace=True, drop=True)
TwoYears_test.reset_index(inplace=True, drop=True)
AllYears_train.reset_index(inplace=True, drop=True)
AllYears_test.reset_index(inplace=True, drop=True)

## OneYear

The code below performs feature selection. I do not recommend to reproduce it, since it takes long (about 2 hours). Hard-coded list of features is copypasted from output.

In [185]:
short_feature_list_defaults = ['Liabilities',
                             'Equity',
                             'Securities',
                             'Investment',
                             'RetainedEarning',
                             'LendCorporate_Me',
                             'LendRetail',
                             'LendInterbank_Me',
                             'BorrowShort_Me',
                             'LendLong_Me',
                             'BondsIssued_Me',
                             'BadCredits_Me',
                             'LendState_Me',
                             'BadBorrowing_Me',
                             'ClientFunds_CBR',
                             'OtherLiabilities_CBR',
                             'Revaluation_CBR',
                             'MainEquity_CBR',
                             'personal_page',
                             'region_notmoscow']

In [189]:
def Gini(y, y_pred):
    res = roc_auc_score(y, y_pred) * 2 - 1
    print(f"Gini: {res}")
    return(res)


In [206]:
estimator=LogisticRegression(penalty="none", max_iter=1000, n_jobs = 4)
estimator.fit(X_train, OneYear_train)
print(f"Accuracy is {estimator.score(X_test, OneYear_test)}")
Gini(list(OneYear_test), estimator.predict_proba(X_test)[:,1])

Accuracy is 0.9044621625402154
Gini: 0.16530299949414418


0.16530299949414418

In [207]:
estimator=LogisticRegression(penalty="none", max_iter=1000, n_jobs = 4)
estimator.fit(X_train[short_feature_list_defaults], OneYear_train)
print(f"Accuracy is {estimator.score(X_test[short_feature_list_defaults], OneYear_test)}")
Gini(list(OneYear_test), estimator.predict_proba(X_test[short_feature_list_defaults])[:,1])

Accuracy is 0.9163519373338929
Gini: 0.739718831106194


0.739718831106194

## TwoYears

We won't retrain feature selection, because it takes too long

In [208]:
estimator=LogisticRegression(penalty="none", max_iter=1000, n_jobs = 4)
estimator.fit(X_train, TwoYears_train)
print(f"Accuracy is {estimator.score(X_test, TwoYears_test)}")
Gini(list(TwoYears_test), estimator.predict_proba(X_test)[:,1])

Accuracy is 0.8345223108127011
Gini: 0.20537851998178436


0.20537851998178436

In [209]:
estimator=LogisticRegression(penalty="none", max_iter=1000, n_jobs = 4)
estimator.fit(X_train[short_feature_list_defaults], TwoYears_train)
print(f"Accuracy is {estimator.score(X_test[short_feature_list_defaults], TwoYears_test)}")
Gini(list(TwoYears_test), estimator.predict_proba(X_test[short_feature_list_defaults])[:,1])

Accuracy is 0.8631976500209819
Gini: 0.7486245671087979


0.7486245671087979

## AllYears

We won't retrain feature selection, because it takes too long

In [211]:
estimator=LogisticRegression(penalty="none", max_iter=1000, n_jobs = 4)
estimator.fit(X_train, AllYears_train)
print(f"Accuracy is {estimator.score(X_test, AllYears_test)}")
Gini(list(AllYears_test), estimator.predict_proba(X_test)[:,1])

Accuracy is 0.7329696461043502
Gini: 0.30553225180842847


0.30553225180842847

In [210]:
estimator=LogisticRegression(penalty="none", max_iter=1000, n_jobs = 4)
estimator.fit(X_train[short_feature_list_defaults], AllYears_train)
print(f"Accuracy is {estimator.score(X_test[short_feature_list_defaults], AllYears_test)}")
Gini(list(AllYears_test), estimator.predict_proba(X_test[short_feature_list_defaults])[:,1])

Accuracy is 0.8178766261015527
Gini: 0.7021217295334676


0.7021217295334676